In [1]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def scrape_polls_to_dataframe(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all tables containing poll data
        tables = soup.find_all('table', class_='wikitable')

        poll_data = []
        header_row = ['Poll source', 'Date administered', 'Democrat', '%', 'Republican', '%', 'Lead margin', 'Sample Size', 'Margin of error', 'state']
        poll_data.append(header_row)


        for table in tables:
            # Extract state name
            state_header = table.find_previous('h3')
            if state_header:
                state = state_header.text
                state = state.replace('[edit]','')
            else:
                continue

            # Extract rows from the table
            t = load_table(table, 'state',state)
            poll_count = 1
            for row in t:
                if row != header_row and len(row) == len(header_row):
                    if poll_count <= 15:
                        poll_data.append(row)
                        poll_count += 1
    
        return poll_data

    else:
        print("Error fetching the webpage. Status code:", response.status_code)
        return None

def load_table(table, descriptor_title = 'none', descriptor = 'none'):
    t = table
    table_data = []
    row_data = []
    rows = t.find_all('tr')
    heads = t.find_all('th')
    heads = [head.text.strip() for head in heads]
    if descriptor_title != 'none':
        heads.append(descriptor_title)
    
    table_data.append(heads)

    # print(heads)

    for row in rows:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]
        if descriptor != 'none':
            cols.append(descriptor)
        if len(cols) > 1:
            table_data.append(cols)         
    return table_data


In [3]:
url = 'https://en.wikipedia.org/wiki/Statewide_opinion_polling_for_the_2012_United_States_presidential_election'
polls = scrape_polls_to_dataframe(url)
polling = pd.DataFrame(polls)

In [4]:
polling.head()
polling.columns = polling.iloc[0]
polling = polling[1:]

In [5]:
polling.loc[polling.state == 'Michigan']

,Poll source,Date administered,Democrat,%,Republican,%,Lead margin,Sample Size,Margin of error,state
140,Mitchell Research & Communications,"November 4, 2012",Obama,51%,Romney,46%,5,"1,305 LV",±2.7%,Michigan
141,Angus Reid Public Opinion,"November 1 – 3, 2012",Obama,52%,Romney,47%,5,502 LV,±4.4%,Michigan
142,Public Policy Polling,"November 1 – 3, 2012",Obama,52%,Romney,46%,6,700 LV,±3.7%,Michigan
143,YouGov,"October 31 – November 3, 2012",Obama,51%,Romney,44%,7,"1,091 LV",±3.3%,Michigan
144,Fox 2 News Detroit/Foster McCollum White & Ass...,"November 2, 2012",Obama,46.24%,Romney,46.86%,0.62,"1,913 LV",±2.24%,Michigan
145,Rasmussen Reports,"November 1, 2012",Obama,52%,Romney,47%,5,750 LV,±4%,Michigan
146,USAction/Project New America/Grove Insight (D),"October 31 – November 1, 2012",Obama,48%,Romney,41%,7,500 LV,±4.4%,Michigan
147,League of Conservation Voters/Public Policy Po...,"October 31 – November 1, 2012",Obama,52%,Romney,46%,6,500 LV,±4.4%,Michigan
148,Health Care for America Now/Public Policy Poll...,"October 30–31, 2012",Obama,53%,Romney,45%,8,500 LV,±4.4%,Michigan
149,Detroit News/Glengariff Group[permanent dead l...,"October 27–29, 2012",Obama,47.7%,Romney,45%,2.7,600 LV,±4%,Michigan


In [6]:
polling.columns.values[3] = 'DEM_PERCENT'
polling.columns.values[5] = 'GOP_PERCENT'

In [7]:
polling['DEM_PERCENT'] = pd.to_numeric(polling['DEM_PERCENT'].str.strip('%')) / 100
polling['GOP_PERCENT'] = pd.to_numeric(polling['GOP_PERCENT'].str.strip('%')) / 100

In [8]:
import duckdb
%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

Deploy FastAPI apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [9]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

In [10]:
%%sql

SELECT 
    2012 as year,
    state,
    COUNT(*) as poll_count, 
    AVG(DEM_PERCENT) as dem_poll_avg,
    AVG(GOP_PERCENT) as gop_poll_avg
FROM polling
GROUP BY state

,year,state,poll_count,dem_poll_avg,gop_poll_avg
0,2012,Arizona,11,0.435455,0.503636
1,2012,Iowa,15,0.482133,0.460333
2,2012,Louisiana,2,0.366000,0.559000
3,2012,Ohio,15,0.492667,0.465333
4,2012,Rhode Island,3,0.565000,0.328000
5,2012,Texas,5,0.394000,0.560000
6,2012,New Jersey,9,0.520000,0.402222
7,2012,Tennessee,4,0.402500,0.535000
8,2012,Wisconsin,15,0.500333,0.456533
9,2012,California,14,0.550214,0.371643


In [11]:
query_output = """
SELECT 
    2012 as year,
    state,
    COUNT(*) as poll_count, 
    AVG(DEM_PERCENT) as dem_poll_avg,
    AVG(GOP_PERCENT) as gop_poll_avg
FROM polling
GROUP BY state
"""

In [12]:
polls_2012 = conn.execute(query_output).fetchall()
polls_2012_df = pd.DataFrame(polls_2012, columns= [col[0] for col in conn.description])

In [13]:
polls_2012_df

,year,state,poll_count,dem_poll_avg,gop_poll_avg
0,2012,Arizona,11,0.435455,0.503636
1,2012,Iowa,15,0.482133,0.460333
2,2012,Louisiana,2,0.366000,0.559000
3,2012,Ohio,15,0.492667,0.465333
4,2012,Rhode Island,3,0.565000,0.328000
5,2012,Texas,5,0.394000,0.560000
6,2012,California,14,0.550214,0.371643
7,2012,Colorado,15,0.478000,0.466667
8,2012,Hawaii,2,0.615000,0.320000
9,2012,Illinois,6,0.566667,0.376667


In [14]:
polls_2012_df.to_csv('C:/Users/appar/OneDrive/jup_nb/polls_2012.csv')